In [5]:
using Revise
using SparseArrays, LinearAlgebra, Statistics, JuMP, ECOS, COSMO
using MathOptInterface, Plots, BenchmarkTools
MOI = MathOptInterface;
using PMPC

In [6]:
M = 1
N, xdim, udim = 30, 2, 1
Nc = 3
x0 = [5.0; 5.0]
fx = PMPC.catb(map(_ -> [1.0 0.1; 0.0 1.0], 1:N)...)
fu = PMPC.catb(map(_ -> [0.0; 1.0][:, :], 1:N)...)
f = PMPC.catb(map(i -> i == 1 ? fx[i] * x0 : zeros(2), 1:N)...)
X_prev, U_prev = randn(xdim, N), randn(udim, N)
X_ref, U_ref = randn(xdim, N), randn(udim, N)
Q = PMPC.catb(map(i -> diagm(0 => (i == N ? [1e0, 1e0] : [1e0, 1.0])), 1:N)...)
q = PMPC.catb(map(_ -> zeros(xdim), 1:N)...)
R = PMPC.catb(map(_ -> diagm(0 => [1e0]), 1:N)...)
r = PMPC.catb(map(_ -> zeros(udim), 1:N)...)
u_limit = 0.4
lu, uu = -u_limit * ones(udim, N), u_limit * ones(udim, N)
add_dim(x, M) = repeat(x; inner=vcat(fill(1, ndims(x)), [M]))
x0 = add_dim(x0, M)
f = add_dim(f, M)
fx = add_dim(fx, M)
fu = add_dim(fu, M)
X_prev = add_dim(X_prev, M)
U_prev = add_dim(U_prev, M)
Q = add_dim(Q, M)
R = add_dim(R, M)
X_ref = add_dim(X_ref, M)
U_ref = add_dim(U_ref, M);
lu, uu = add_dim(lu, M), add_dim(uu, M);
args = x0, f, fx, fu, X_prev, U_prev, Q, R, X_ref, U_ref;

In [7]:
X, U = randn(xdim, N), randn(udim, N)

([-0.10568417444405062 -0.44068361695468034 … 2.1724048848926736 1.2687150100923779; 0.10902816246746086 1.540194651951932 … 0.2883369813691004 -1.354427116370301], [-1.4562211949674737 -0.9340658587651888 … -0.20801337019339003 0.36382101438744513])

In [18]:
probs = PMPC.make_probs(x0, f, fx, fu, X_prev, U_prev, Q, R, X_ref, U_ref; reg_x=1.0, reg_u=2.0, slew_reg=0.0)
P, q, resid = PMPC.qp_repr_Pq(probs[1])

(sparse([1, 2, 1, 2, 3, 2, 3, 4, 3, 4  …  81, 82, 83, 84, 85, 86, 87, 88, 89, 90], [1, 1, 2, 2, 2, 3, 3, 3, 4, 4  …  81, 82, 83, 84, 85, 86, 87, 88, 89, 90], [103.0, -100.0, -100.0, 203.0, -100.0, -100.0, 203.0, -100.0, -100.0, 203.0  …  2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0], 90, 90), [-3.0411548426770887, -0.555999435405525, 1.3076915666277862, -0.20009950156107337, 0.5604020527972817, -0.8497906360618175, -0.3999139412957926, 0.634876926256306, -0.8099695240031338, -1.2983104463368074  …  -0.2959762891783562, 1.2446680173966906, 0.9322221787440785, -1.1070190112947573, -0.8859311104867418, -0.4944757146058296, 2.1590446716334784, 2.3246043297625247, -0.15735254918044173, -0.12212320366465096], 91.99926284398387)

In [82]:
function objective(X, U, X_prev, U_prev, reg_x, reg_u, Q, R, X_ref, U_ref, P, q, resid)
    obj = 0.0
    @views for i in 1:size(Q, 3)
        dx = X[:, i] - X_ref[:, i]
        du = U[:, i] - U_ref[:, i]
        obj += 0.5 * dot(dx, Q[:, :, i], dx) + 0.5 * dot(du, R[:, :, i], du)
    end
    @views for i in 1:size(Q, 3)
        dx = X[:, i] - X_prev[:, i]
        du = U[:, i] - U_prev[:, i]
        obj += 0.5 * reg_x * dot(dx, dx) + 0.5 * reg_u * dot(du, du)
    end
    z = vcat(reshape(U, :), reshape(X, :))
    obj2 = 0.5 * dot(z, P, z) + dot(q, z) + resid
    return obj, obj2
end
objective(X, U, X_prev[:, :, 1], U_prev[:, :, 1], probs[1].reg_x, probs[1].reg_u, Q[:, :, :, 1], R[:, :, :, 1], X_ref[:, :, 1], U_ref[:, :, 1], P, q, resid)

(133.96216878446054, 133.9621687844606)

In [20]:
println(nnz(P))
println(nnz(sparse(cholesky(P).L)))

148
119


In [30]:
probs = PMPC.make_probs(x0, f, fx, fu, X_prev, U_prev, Q, R, X_ref, U_ref; reg_x=1.0, reg_u=2.0, slew_reg=1e2)
P, q, resid = PMPC.qp_repr_Pq(probs[1])

(sparse([1, 2, 1, 2, 3, 2, 3, 4, 3, 4  …  81, 82, 83, 84, 85, 86, 87, 88, 89, 90], [1, 1, 2, 2, 2, 3, 3, 3, 4, 4  …  81, 82, 83, 84, 85, 86, 87, 88, 89, 90], [103.0, -100.0, -100.0, 203.0, -100.0, -100.0, 203.0, -100.0, -100.0, 203.0  …  2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0], 90, 90), [-3.0411548426770887, -0.555999435405525, 1.3076915666277862, -0.20009950156107337, 0.5604020527972817, -0.8497906360618175, -0.3999139412957926, 0.634876926256306, -0.8099695240031338, -1.2983104463368074  …  -0.2959762891783562, 1.2446680173966906, 0.9322221787440785, -1.1070190112947573, -0.8859311104867418, -0.4944757146058296, 2.1590446716334784, 2.3246043297625247, -0.15735254918044173, -0.12212320366465096], 91.99926284398387)

In [31]:
println(nnz(P))
println(nnz(sparse(cholesky(P).L)))

148
119


In [32]:
help(cholesky)

UndefVarError: UndefVarError: help not defined
  Welcome to Julia 1.8.5. The full manual is available at

  https://docs.julialang.org

  as well as many great tutorials and learning resources:

  https://julialang.org/learning/

  For help on a specific function or macro, type ? followed by its name, e.g.
  ?cos, or ?@time, and press enter. Type ; to enter shell mode, ] to enter
  package mode.

  To exit the interactive session, type CTRL-D (press the control key together
  with the d key), or type exit().

In [33]:
F = cholesky(P)
L = sparse(sparse(F.L)[sortperm(F.p), :]') / sqrt(2.0)

90×90 SparseMatrixCSC{Float64, Int64} with 119 stored entries:
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣠⠞⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⣠⠏⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⣠⠞⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⣠⠏⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⣠⠞⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⣠⠏⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠺⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄

In [34]:
nnz(L)

119

In [3]:
solver_settings = Dict(:verbose => true)
settings = Dict(
  :verbose => true,
  :Nc => Nc,
  :solver => "cosmo",
  :lu => lu,
  :uu => uu,
  :smooth_cstr => "squareplus",
  :smooth_alpha => 1e1,
  :smooth_beta => 1e4,
  :solver_settings => solver_settings,
)
#@time X1, U1, data = PMPC.lsocp_solve(args...; settings..., solver="cosmo");
@time X2, U2, data = PMPC.lsocp_solve(args...; settings..., solver="ecos");
@time X3, U3, data = PMPC.lsocp_solve(args...; settings..., solver="jump");
println(norm(X2 - X3))

SOCP_k = 12
 15.744251 seconds (73.10 M allocations: 3.657 GiB, 5.75% gc time, 98.95% compilation time)


SOCP_k = 12



ECOS 2.0.8 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  -1.308e+06  -1.308e+06  +1e+07  3e-01  3e-05  1e+00  2e+04    ---    ---    1  1  - |  -  - 
 1  -1.166e+06  -1.165e+06  +7e+06  3e-03  2e-05  1e+03  1e+04  0.4874  1e-01   3  4  3 |  0  0
 2  -1.138e+06  -1.136e+06  +7e+06  2e-03  2e-05  2e+03  1e+04  0.0765  5e-01   3  3  2 |  0  0
 3  -9.137e+05  -9.118e+05  +4e+06  7e-04  9e-06  2e+03  6e+03  0.5218  2e-01   3  3  3 |  0  0
 4  -4.179e+05  -4.176e+05  +7e+05  1e-04  1e-06  4e+02  1e+03  0.8347  1e-02   3  3  3 |  0  0
 5  -3.733e+05  -3.729e+05  +7e+05  8e-05  1e-06  4e+02  1e+03  0.1556  7e-01   4  4  3 |  0  0
 6  -9.597e+04  -9.572e+04  +3e+05  3e-05  5e-07  2e+02  4e+02  0.7271  2e-01   4  3  4 |  0  0
 7  -2.216e+04  -2.201e+04  +1e+05  1e-05  2e-07  2e+02  2e+02  0.6395  2e-01   4  3  4 |  0  0
 8  -2.039e+04  -2.022e+04  +1e+05  1e-05  2e-

0.004548865303767811
   4  3  3 |  0  0
13  +6.065e+04  +6.065e+04  +3e+02  4e-08  6e-10  1e+00  4e-01  0.9175  1e-01   4  2  2 |  0  0
14  +6.088e+04  +6.088e+04  +1e+02  2e-08  3e-10  5e-01  2e-01  0.8345  3e-01   3  2  2 |  0  0
15  +6.109e+04  +6.109e+04  +2e+01  2e-08  4e-11  6e-02  2e-02  0.9890  1e-01   4  2  2 |  0  0
16  +6.112e+04  +6.112e+04  +4e+00  8e-09  9e-12  1e-02  5e-03  0.9890  2e-01   4  2  2 |  0  0
17  +6.113e+04  +6.113e+04  +4e-01  4e-09  5e-12  1e-03  6e-04  0.9762  6e-02   3  2  2 |  0  0
18  +6.113e+04  +6.113e+04  +2e-01  2e-09  2e-12  6e-04  3e-04  0.9108  3e-01   3  2  2 |  0  0
19  +6.113e+04  +6.113e+04  +3e-02  1e-09  2e-12  1e-04  5e-05  0.9517  7e-02   4  2  2 |  0  0
20  +6.113e+04  +6.113e+04  +1e-02  8e-10  6e-13  4e-05  2e-05  0.9890  2e-01   3  1  2 |  0  0
21  +6.113e+04  +6.113e+04  +3e-03  5e-10  4e-13  9e-06  4e-06  0.9827  1e-01   3  1  1 |  0  0
22  +6.113e+04  +6.113e+04  +3e-03  5e-10  4e-13  7e-06  4e-06  0.9890  5e-01   2  1  2 |  0  0


In [34]:
U2[1, :, 1]

30-element Vector{Float64}:
 -0.39999999557591914
 -0.39999999598488667
 -0.3999999963491315
 -0.39999999665230035
 -0.39999999666583796
 -0.3999999966534288
 -0.3999999966077374
 -0.3999999965187852
 -0.3999999963727377
 -0.3999999961499855
  ⋮
 -0.12859585243741484
  0.39998588415064146
  0.3999993480975383
  0.3999995724412849
  0.3999996151051727
  0.3999995846866195
  0.39999945876642834
  0.39999898035530146
  0.3643831285213187

In [13]:
norm(X2 - X3)

4.472329316084468e-6

In [80]:
println(repeat('#', 80))
@time X4, U4, data = PMPC.lqp_solve(args...; settings..., solver="mosek");

PMPC.JuMPSolver{Float64}(nothing, sparse([1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  1038, 1039, 1040, 1041, 1042, 1043, 1044, 1045, 1046, 1047], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  1038, 1039, 1040, 1041, 1042, 1043, 1044, 1045, 1046, 1047], [12.0, 12.0, 12.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 1047, 1047), [0.0, 0.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0  …  -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0], sparse([1, 2, 61, 62, 121, 122, 181, 182, 241, 242  …  717, 718, 717, 719, 720, 718, 719, 720, 719, 720], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1043, 1043, 1044, 1044, 1044, 1045, 1045, 1045, 1046, 1047], [0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0  …  0.1, 1.0, -1.0, 1.0, 0.0, -1.0, 0.1, 1.0, -1.0, -1.0], 720, 1047), [-5.0, -5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], sparse([1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  318, 319, 320, 321, 322, 323, 324, 325, 326, 327], [1

################################################################################


Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 3731            
  Cones                  : 655             
  Scalar variables       : 4713            
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimizat

  4.045401 seconds (10.86 M allocations: 644.159 MiB, 2.38% gc time, 95.97% compilation time: 2% of which was recompilation)


In [7]:
p = plot()
for i in 1:udim
    plot!(p, U2[i, :, 1])
end

In [8]:
U2[1, :, 1]

30-element Vector{Float64}:
 -2.999999999749112
 -2.9999999991196784
 -2.323219411135405
 -0.6075464367246329
  0.02408429869050157
  0.2421343216030011
  0.3039139059518766
  0.30787761088133514
  0.29158926506005906
  0.2692812149861184
  ⋮
  0.09497295956671387
  0.08881068442028751
  0.08351383801202387
  0.07897820627893869
  0.07502581562463062
  0.07127240394266827
  0.06678274975014696
  0.05917268294310772
  0.04226797185182386

In [21]:
@btime PMPC.lsocp_solve(
  x0,
  f,
  fx,
  fu,
  X_prev,
  U_prev,
  Q,
  R,
  X_ref,
  U_ref;
  solver="cosmo",
  solver_settings=solver_settings,
);


  89.011 ms (133977 allocations: 169.12 MiB)


In [ ]:
@btime PMPC.lsocp_solve(
  x0,
  f,
  fx,
  fu,
  X_prev,
  U_prev,
  Q,
  R,
  X_ref,
  U_ref;
  solver="ecos",
  solver_settings=solver_settings,
);


# Test logbarrier constraints


In [66]:
A = [
  0 1.0
  1.0 0
  0 -1.0
  -1.0 0
]
b = [1.0, 1, 1, 1]
G_left, G_right, h = PMPC.smoothen_linear_inequlities(sparse(A), b, 1.0)



ECOS 2.0.8 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  -4.000e+00  -4.040e+00  +4e-02  5e-03  5e-03  1e+00  2e-01    ---    ---    0  1  - |  -  - 
 1  -4.000e+00  -4.000e+00  +5e-04  6e-05  6e-05  1e-02  2e-03  0.9890  1e-04   1  1  1 |  0  0
 2  -4.000e+00  -4.000e+00  +7e-06  6e-07  6e-07  1e-04  3e-05  0.9890  1e-04   1  1  1 |  0  0
 3  -4.000e+00  -4.000e+00  +8e-08  7e-09  7e-09  1e-06  3e-07  0.9890  1e-04   1  0  0 |  0  0
 4  -4.000e+00  -4.000e+00  +1e-09  8e-11  8e-11  1e-08  3e-09  0.9890  1e-04   1  0  0 |  0  0

OPTIMAL (within feastol=7.6e-11, reltol=2.6e-10, abstol=1.0e-09).
Runtime: 0.000066 seconds.



(sparse([5, 11, 2, 8], [1, 1, 2, 2], [-1.0, 1.0, -1.0, 1.0], 12, 2), sparse([1, 4, 7, 10], [1, 2, 3, 4], [-1.0, -1.0, -1.0, -1.0], 12, 4), [0.0, -1.0, -1.0, 0.0, -1.0, -1.0, 0.0, -1.0, -1.0, 0.0, -1.0, -1.0])

In [67]:
prob = PMPC.ConeProblem(0, Int[], 4, G, nothing, vcat(randn(2), ones(4)), h, nothing)
#prob = PMPC.ConeProblem(4, Int[], 0, A, nothing, randn(2), b, nothing)


PMPC.ConeProblem(0, Int64[], 4, sparse([5, 11, 2, 8, 1, 4, 7, 10], [1, 1, 2, 2, 3, 4, 5, 6], [-1.0, 1.0, -1.0, 1.0, -1.0, -1.0, -1.0, -1.0], 12, 6), nothing, [-0.6099963913176603, 0.6020823756232077, 1.0, 1.0, 1.0, 1.0], [0.0, -1.0, -1.0, 0.0, -1.0, -1.0, 0.0, -1.0, -1.0, 0.0, -1.0, -1.0], nothing)

In [68]:
sol_ecos = PMPC.ECOS_solve(prob)
sol_cosmo = PMPC.COSMO_solve(prob)
model = Model(() -> ECOS.Optimizer())
x = @variable(model, x[1:2])
t = @variable(model, t[1:4])
all = @expression(model, [x; t])
for i in 1:div(size(G, 1), 3)
  st = 3 * (i - 1) + 1
  @constraint(
    model,
    [G[st, :]' * all - h[st], G[st + 2, :]' * all - h[st + 2], G[st + 1, :]' * all - h[st + 1]] in
    MOI.ExponentialCone()
  )
end
@objective(model, Min, prob.c' * all)
optimize!(model)
sol_model = (x=value.(all),)



ECOS 2.0.8 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +0.000e+00  -3.631e+00  +1e+01  5e-01  3e-01  1e+00  1e+00    ---    ---    0  0  - |  -  - 
 1  -2.117e+00  -3.118e+00  +3e+00  2e-01  9e-02  5e-01  2e-01  0.7833  9e-03   1  1  1 |  1  1
 2  -2.455e-01  -4.906e-01  +5e-01  9e-02  2e-02  2e-01  4e-02  0.7833  9e-03   1  1  1 |  1  1
 3  -2.678e-01  -3.223e-01  +1e-01  2e-02  5e-03  4e-02  1e-02  0.7833  1e-02   1  1  1 |  1  1
 4  -1.806e-01  -1.934e-01  +2e-02  5e-03  1e-03  1e-02  2e-03  0.7833  9e-03   1  1  1 |  1  1
 5  -1.811e-01  -1.839e-01  +6e-03  1e-03  3e-04  2e-03  5e-04  0.7833  2e-02   1  1  1 |  1  1
 6  -1.762e-01  -1.769e-01  +1e-03  3e-04  7e-05  5e-04  1e-04  0.7833  9e-03   2  1  1 |  1  1
 7  -1.764e-01  -1.765e-01  +3e-04  6e-05  1e-05  1e-04  3e-05  0.7833  2e-02   1  1  1 |  1  1
 8  -1.761e-01  -1.761e-01  +6e-05  1e-05  3e-

(x = [0.2809276385176336, -0.27780770018368073, -0.24514587394355586, 0.32979328185418, 0.32546382975480354, -0.24758453172709655],)

In [79]:
function fn(x)
  alf = 1e0
  return prob.c[1:2]' * x + sum(-log.(-alf * (A * x - b)) / alf)
end
g_fn(x) = gradient(fn, x)[1]


g_fn (generic function with 1 method)

In [89]:
x = 1e-2 * ones(2)
for it in 1:100000
  x = x - 1e-3 * g_fn(x)
end
println(x)


[0.28092763546568544, -0.27780769698698704]


In [90]:
hcat(sol_ecos.x, sol_cosmo.x, sol_model.x, prob.c)


6×4 Matrix{Float64}:
  0.280928   0.29588    0.280928  -0.609996
 -0.277808  -0.292265  -0.277808   0.602082
 -0.245146   0.331332  -0.245146   1.0
  0.329793  -0.24701    0.329793   1.0
  0.325464  -0.244654   0.325464   1.0
 -0.247585   0.335879  -0.247585   1.0

# Test SOC constraints


In [46]:
P, q = randn(4, 4), randn(4)
P = P * P'
G_left, G_right, h = PMPC.Pqr2Gh(P, q)
G = hcat(G_left, G_right)
cone_problem = PMPC.ConeProblem(0, [size(G, 1)], 0, G, nothing, vcat(zeros(4), ones(1)), h, nothing)
sol_ecos = PMPC.ECOS_solve(cone_problem)
sol_cosmo = PMPC.COSMO_solve(cone_problem)
model = Model(() -> ECOS.Optimizer())
x = @variable(model, x[1:4])
t = @variable(model, t)
all = @expression(model, [x; t])
@constraint(model, G * all - h in MOI.SecondOrderCone(length(h)))
@objective(model, Min, t)
optimize!(model)
sol_model = (x=value.(all),)



ECOS 2.0.8 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  -1.073e+02  -1.278e+02  +8e-02  1e-04  6e-02  1e+00  5e-01    ---    ---    1  1  - |  -  - 
 1  -1.073e+02  -1.075e+02  +2e-03  1e-06  8e-04  1e-02  6e-03  0.9890  1e-04   1  1  1 |  0  0
 2  -1.073e+02  -1.073e+02  +2e-05  1e-08  9e-06  1e-04  7e-05  0.9890  1e-04   1  1  1 |  0  0
 3  -1.073e+02  -1.073e+02  +2e-07  1e-10  1e-07  1e-06  7e-07  0.9890  1e-04   1  1  1 |  0  0
 4  -1.073e+02  -1.073e+02  +3e-09  1e-12  1e-09  1e-08  8e-09  0.9890  1e-04   1  1  1 |  0  0

OPTIMAL (within feastol=1.1e-09, reltol=2.4e-11, abstol=2.6e-09).
Runtime: 0.000087 seconds.


ECOS 2.0.8 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  -1.073e+02  -1.278e+02  +8e-02  1e-04  6e-02  1e+0

(x = [53.33454435984867, 56.03560777076323, -20.720844365600016, -86.29104476011261, -107.28841321620486],)

In [47]:
hcat(sol_ecos.x, sol_cosmo.x, vcat(P \ -q, zeros(1)), sol_model.x)


5×4 Matrix{Float64}:
   53.3345    53.3346   53.3346    53.3345
   56.0356    56.0356   56.0356    56.0356
  -20.7208   -20.7209  -20.7209   -20.7208
  -86.291    -86.2911  -86.2911   -86.291
 -107.288   -107.288     0.0     -107.288

# Test subcomponents


In [7]:
P, q = randn(5, 5), randn(5)
P = P * P'
G_left, G_right, h = PMPC.Pqr2Gh(P, q)


┌ Warning: Error in P reconstruction is 1.5423711185402538e-15
└ @ PMPC /home/rdyro/Dropbox/stanford/sensitivity_analysis/pmpc/PMPC.jl/src/socp.jl:145


[-119.35138382405168, -123.0048713967983, 148.7290648592994, 210.5446486412796, -72.57508855539143]



([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.2368746655669338 0.633398034401705; 0.0 0.0 … 0.0 0.18756490917521243],   [1]  =  1.0
  [2]  =  1.0, [-201.00623569299762, -200.50623569299762, 0.15709384102398355, -0.43558605604391804, -0.03349626382741731, 3.903774796296084, -13.612539893274992])

In [69]:
model = Model(() -> ECOS.Optimizer())
t = @variable(model, t)
x = @variable(model, x[1:size(P, 1)])
G = hcat(G_left, G_right)
f = @expression(model, G * [x; t] - h)
@constraint(model, f in SecondOrderCone())
@objective(model, Min, t)
optimize!(model)



ECOS 2.0.8 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  -2.008e+02  -2.296e+02  +5e+00  4e-03  5e-02  1e+00  3e+00    ---    ---    1  1  - |  -  - 
 1  -2.008e+02  -2.013e+02  +8e-02  7e-05  8e-04  2e-02  5e-02  0.9813  1e-03   1  1  1 |  0  0
 2  -2.008e+02  -2.008e+02  +1e-03  8e-07  1e-05  2e-04  9e-04  0.9870  3e-04   1  1  1 |  0  0
 3  -2.008e+02  -2.008e+02  +2e-05  9e-09  1e-07  2e-06  1e-05  0.9890  1e-04   1  1  1 |  0  0
 4  -2.008e+02  -2.008e+02  +2e-07  1e-10  1e-09  3e-08  1e-07  0.9890  1e-04   1  1  1 |  0  0

OPTIMAL (within feastol=1.2e-09, reltol=8.8e-10, abstol=1.8e-07).
Runtime: 0.000067 seconds.



In [70]:
hcat(value.(x), P \ -q)


5×2 Matrix{Float64}:
 -119.351  -119.351
 -123.005  -123.005
  148.729   148.729
  210.545   210.545
  -72.575   -72.5751

In [54]:
value.(t)


-200.75623569299765

In [56]:
G * value.([x; t]) - h


7-element Vector{Float64}:
  0.24999999999997158
 -0.2500000000000284
 -3.647082635893639e-14
 -2.015054789694659e-14
  6.492723025885994e-14
 -2.886579864025407e-14
 -3.552713678800501e-15

In [57]:
hcat(hcat(G_left, G_right)[2:end, :] * [JuMP.value(t); JuMP.value.(x)], h[2:end])


6×2 Matrix{Float64}:
  -72.5751  -200.506
  -97.942      0.157094
 -247.242     -0.435586
 -375.844     -0.0334963
  168.589      3.90377
   39.4908   -13.6125

In [39]:
hcat(G_left, G_right)


7×6 SparseMatrixCSC{Float64, Int64} with 17 stored entries:
  ⋅        ⋅          ⋅          ⋅           ⋅        1.0
  ⋅        ⋅          ⋅          ⋅           ⋅        1.0
 1.67136  0.547229  -0.231552   0.159334    0.216116   ⋅ 
  ⋅       0.57362   -0.42962    0.0497826  -0.279794   ⋅ 
  ⋅        ⋅         0.888342  -1.12844     0.909855   ⋅ 
  ⋅        ⋅          ⋅         0.358421   -0.123254   ⋅ 
  ⋅        ⋅          ⋅          ⋅          0.400572   ⋅ 

In [21]:
xs = -P \ q


5-element Vector{Float64}:
 -2.556075335970547
  9.76024843922445
 12.745812333491871
  8.56963253189468
 -0.707712315051311

In [52]:
eigs(P)


UndefVarError: UndefVarError: eigs not defined

In [65]:
F = cholesky(P)


Cholesky{Float64, Matrix{Float64}}
U factor:
5×5 UpperTriangular{Float64, Matrix{Float64}}:
 1.80014  1.29524  1.55147   1.28738   -0.0711716
  ⋅       1.31349  1.64502   0.186696  -0.791152
  ⋅        ⋅       1.41837  -1.16538   -2.71596
  ⋅        ⋅        ⋅        0.317638  -0.506312
  ⋅        ⋅        ⋅         ⋅         0.0742055

In [1]:
F.L * F.L' - P


UndefVarError: UndefVarError: F not defined